<a href="https://colab.research.google.com/github/uzi69-158-bse/BUS-RESERVATION/blob/main/LANGCHAIN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introduction to Developing Applications with LangChain

Key Ideas:

LangChain is a framework that simplifies the development of applications using large language models (LLMs).
It provides a structured way to build applications by composing modular components like chains, agents, and prompt templates.
In Google Colab, you can quickly prototype with free tools and models (e.g., Hugging Face models).

In [ ]:
!pip install langchain transformers


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


#2. Overview of LangChain and Its Ecosystem


Key Ideas:

Ecosystem Components: LangChain’s ecosystem includes chains (sequential steps), prompt templates, memory, and agents.

Free Resources: You can leverage Hugging Face’s free models alongside LangChain components for building applications.

Integration: It unifies different LLMs and makes switching between models seamless.

Remember: While some providers (like OpenAI) require API keys or paid plans, many free alternatives (e.g., Hugging Face’s GPT-2 or BLOOM) are available.

#3. Core Components of LangChain


Components to Master:

Chains: A sequence of operations or steps that process input and produce output.

Prompt Templates: Reusable templates that format input to the LLM.

Memory: Modules that help maintain conversational or application context.

Agents: Components that dynamically decide which actions or chains to run based on user input.

In [ ]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Load a better model for instruction-following
generator = pipeline('text-generation', model='tiiuae/falcon-7b-instruct')

# Define an LLM model
hf_llm = HuggingFacePipeline(pipeline=generator)

# Define the prompt template
template = "Rewrite history: {text}"
prompt_template = PromptTemplate(input_variables=["text"], template=template)

# Create the chain
chain = LLMChain(llm=hf_llm, prompt=prompt_template)

# Invoke the chain
result = chain.invoke({"text": "The Moon Landing of 1969"})
print(result)


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

KeyboardInterrupt: 

#4. Using Hugging Face and OpenAI Models with LangChain

Key Ideas:

Hugging Face Models: They are free and accessible; you can use models like GPT-2 or BLOOM without an API key.

OpenAI Models: While popular, they typically require an API key and paid subscription—so for free development, stick with Hugging Face.

In [ ]:
# Already installed in previous cell
from transformers import pipeline

# Use a free text-generation model from Hugging Face
generator = pipeline("text-generation", model="gpt2")
prompt = "Once upon a time"
output = generator(prompt, max_length=50)
print(output[0]['generated_text'])


#5. Unifying Different Models with LangChain


Key Ideas:

Abstraction Layer: LangChain allows you to wrap different models (even from different providers) behind a common interface.

Interoperability: Easily switch between models by changing the underlying function, while reusing your prompt templates and chains.

Tip: Focus on building your chains so that your application logic is decoupled from the specific model—this makes upgrades or changes easier.


In [ ]:
def get_model(provider="openai"):
    if provider == "openai":
        return OpenAI(model_name="text-davinci-003", temperature=0.7)
    elif provider == "huggingface":
        return HuggingFaceHub(repo_id="bigscience/bloom", model_kwargs={"temperature": 0.7})
    else:
        raise ValueError("Unsupported provider")

def run_chain(model_provider, query):
    model = get_model(model_provider)
    chain = LLMChain(llm=model, prompt=PromptTemplate.from_template("Explain {query} in simple terms."))
    return chain.run(query)

# Switching between models
print(run_chain("openai", "Quantum Computing"))
print(run_chain("huggingface", "Quantum Computing"))


#6. Implementing Prompting Strategies for Chatbots

Key Ideas:

Prompting: The art of designing input text that guides the LLM’s output effectively.

Strategies: Include zero-shot, few-shot, and chain-of-thought prompting.

Practice: Experiment by creating variations of prompts to see how output quality changes.

In [ ]:
#zero shot prompting
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-4", temperature=0.7)

prompt = "Explain the concept of reinforcement learning in simple terms."
response = llm.predict(prompt)
print(response)


In [ ]:
#few shot prompting
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Few-shot examples in prompt
few_shot_prompt = """Q: What is the capital of France?
A: The capital of France is Paris.

Q: What is the capital of Japan?
A: The capital of Japan is Tokyo.

Q: What is the capital of Germany?
A: """

llm = OpenAI(model_name="gpt-4", temperature=0.5)
response = llm.predict(few_shot_prompt)
print(response)  # Expected output: "The capital of Germany is Berlin."




In [ ]:
#chain of thought(CoT) prompting
cot_prompt = """Q: A farmer has 3 chickens, 2 cows, and 4 horses. How many total legs do the animals have?
A: Let's think step by step.
- Chickens have 2 legs each, so 3 × 2 = 6 legs.
- Cows have 4 legs each, so 2 × 4 = 8 legs.
- Horses have 4 legs each, so 4 × 4 = 16 legs.
- Adding them together: 6 + 8 + 16 = 30.
So, the total number of legs is: """

llm = OpenAI(model_name="gpt-4", temperature=0.5)
response = llm.predict(cot_prompt)
print(response)  # Expected output: "30."


In [ ]:
#dynamic prompting in chatbots
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

chatbot = ConversationChain(llm=ChatOpenAI(model_name="gpt-4", temperature=0.7))

print(chatbot.predict("How does photosynthesis work?"))
print(chatbot.predict("Explain it like I’m five."))


#7. Using Prompt Templates in LangChain


Key Ideas:

Reusable Templates: Define a structure that you can reuse for multiple inputs.

Flexibility: Templates can include variables to be filled in at runtime.

In [ ]:
from langchain.prompts import PromptTemplate

template = "Summarize the following text in one sentence:\n\n{text}"
prompt = PromptTemplate(input_variables=["text"], template=template)
formatted_prompt = prompt.format(text="Artificial Intelligence is transforming the world.")
print(formatted_prompt)


Summarize the following text in one sentence:

In an alternate timeline, the Apollo 11 mission was not just an American achievement but a global collaboration. Instead of a Cold War-driven space race, nations worldwide united under the banner of scientific exploration, forming the United Space Federation (USF) in 1965. This initiative brought together the best minds from the United States, Soviet Union, Europe, and beyond, pooling resources and technology to achieve one singular goal: humanity's first step beyond Earth.

By July 20, 1969, the world watched in anticipation as the USF Lunar Module "Unity" descended onto the Moon's surface. Instead of the American flag, the astronauts—Neil Armstrong, Alexei Leonov (a Soviet cosmonaut), and Haruto Takahashi (a Japanese engineer)—planted a blue flag with a golden planet symbol, representing humanity's unity in exploration.

As Armstrong took the historic step, his words were not about American pride, but a message to all of humanity:


#8. Chat Models and Chat Prompt Templates


Key Ideas:

Chat Models: Specialize in conversational tasks.

Chat Prompts: Are designed to simulate dialogue, often using system and user message structures.

Example Concept:
While the previous examples use text-generation models, you could mimic chat-like interactions by formatting prompts that alternate between “User:” and “Assistant:” lines.

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Define structured chat messages
system_prompt = SystemMessagePromptTemplate.from_template("You are a helpful assistant that answers questions concisely.")
user_prompt = HumanMessagePromptTemplate.from_template("What is the largest planet in our solar system?")

# Create Chat Prompt Template
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# Format and print the chat prompt
formatted_prompt = chat_prompt.format_messages()
print(formatted_prompt)


#9. Few-Shot Prompting in LangChain


Key Ideas:

Few-Shot Learning: Provide the LLM with a few examples within the prompt so that it “learns” the pattern.

Application: Especially useful when you need the model to follow a specific format.

In [ ]:
few_shot_template = """
Translate English to French:
Example 1: English: "Good morning", French: "Bonjour"
Example 2: English: "Thank you", French: "Merci"
Now, translate English: "{text}"
"""

prompt = PromptTemplate(input_variables=["text"], template=few_shot_template)
formatted_prompt = prompt.format(text="I love learning new things")
print(formatted_prompt)


#10. Implementing Sequential Chains


Key Ideas:

Sequential Execution: Chains can be set up to perform multiple operations in sequence (e.g., cleaning input, generating a response, post-processing).

Benefits: This modular approach makes debugging and development easier.

Example Concept:
Set up one chain to format the prompt and another to generate text, then connect them so that the output of one becomes the input of the next.

In [ ]:
#basic sequential chain
from langchain.chains import SimpleSequentialChain
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-4", temperature=0.7)

# First chain: Reformats user input into a structured prompt
def format_prompt(input_text):
    return f"Provide a summary of the following text:\n\n{input_text}"

# Second chain: Generates a response based on the formatted prompt
response_chain = SimpleSequentialChain(
    chains=[format_prompt, llm],
    verbose=True
)

input_text = "LangChain is a framework for developing applications powered by large language models."
result = response_chain.run(input_text)
print(result)


In [ ]:
"""using LangChain's SequentialChain for Multiple Steps
A SequentialChain allows multiple chains with defined inputs and outputs.

Example: Multi-Step Processing
Step 1: Clean and reformat the input.
Step 2: Generate an AI-based response.
Step 3: Post-process the response (e.g., summarize)."""

from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate

# Initialize LLM
llm = OpenAI(model_name="gpt-4", temperature=0.7)

# First chain: Cleaning and structuring user input
input_prompt = PromptTemplate(
    input_variables=["text"],
    template="Reformat this text for clarity:\n{text}"
)
cleaning_chain = LLMChain(llm=llm, prompt=input_prompt, output_key="cleaned_text")

# Second chain: Generating a response based on cleaned text
response_prompt = PromptTemplate(
    input_variables=["cleaned_text"],
    template="Provide a detailed analysis:\n{cleaned_text}"
)
response_chain = LLMChain(llm=llm, prompt=response_prompt, output_key="analysis")

# Third chain: Summarizing the response
summary_prompt = PromptTemplate(
    input_variables=["analysis"],
    template="Summarize the key points:\n{analysis}"
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")

# Create the sequential chain
seq_chain = SequentialChain(
    chains=[cleaning_chain, response_chain, summary_chain],
    input_variables=["text"],
    output_variables=["summary"],
    verbose=True
)

input_text = "LangChain allows developers to build powerful applications with large language models."
result = seq_chain({"text": input_text})
print(result["summary"])
#✅ Best for: Complex workflows like preprocessing → response generation → post-processing.

In [ ]:
from langchain.schema.runnable import RunnableLambda
from textblob import TextBlob
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="gpt-4", temperature=0.7)

# Step 1: Perform sentiment analysis using a function
def analyze_sentiment(text):
    sentiment = TextBlob(text).sentiment.polarity
    return {"sentiment": "positive" if sentiment > 0 else "negative" if sentiment < 0 else "neutral"}

sentiment_chain = RunnableLambda(analyze_sentiment)

# Step 2: Generate a response based on sentiment
sentiment_prompt = PromptTemplate(
    input_variables=["sentiment"],
    template="Write a message based on this sentiment: {sentiment}"
)
response_chain = LLMChain(llm=llm, prompt=sentiment_prompt, output_key="message")

# Step 3: Chain them together
sentiment_seq_chain = SequentialChain(
    chains=[sentiment_chain, response_chain],
    input_variables=["text"],
    output_variables=["message"],
    verbose=True
)

# Test input
input_text = "I love using LangChain; it's amazing!"
result = sentiment_seq_chain({"text": input_text})
print(result["message"])


#11. Introduction to Agents in LangChain


Key Ideas:

Agents: More dynamic than chains, agents can decide which action to take next based on the conversation or task.

Usage: Ideal for complex applications where decisions need to be made on the fly.

Tip: Start with simple decision-making agents before moving to more complex scenarios.

Agents operate by:

✅ Receiving input from a user

✅ Using tools (like search, API calls, or databases) to
gather information

✅ Deciding which action to take next

✅ Producing a final response

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import Tool

# Initialize LLM
llm = OpenAI(model_name="gpt-4", temperature=0.7)

# Define a simple tool (a function that the agent can use)
def square_number(n: str):
    return int(n) ** 2

square_tool = Tool(
    name="SquareNumber",
    func=square_number,
    description="Calculates the square of a given number."
)

# Initialize an agent with a tool
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Simple decision-making agent
    tools=[square_tool],  # List of tools the agent can use
    llm=llm,
    verbose=True
)

# Run agent with a prompt
response = agent.run("What is the square of 8?")
print(response)


In [ ]:
from langchain.tools import tool
from langchain.utilities import SerpAPIWrapper

# Define multiple tools
@tool
def multiply_numbers(a: str, b: str):
    """Multiplies two numbers."""
    return int(a) * int(b)

# Web search tool
search = SerpAPIWrapper()

# Initialize agent with multiple tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[multiply_numbers, search],
    llm=llm,
    verbose=True
)

# Run the agent
response = agent.run("What is 12 multiplied by 5, and what is the capital of France?")
print(response)


#12. Understanding React Agents


Key Ideas:

ReAct Framework: Combines reasoning (think) and acting (execute) in a loop, letting agents plan and then perform actions.

How It Works: The agent receives input, reasons about it, and then decides which tool or chain to execute next.

Tip: Familiarize yourself with the “ReAct” pattern as it’s a powerful way to design interactive applications.

The ReAct Agent follows these steps:

1️⃣ Receives input from the user.

2️⃣ Thinks about the best approach using reasoning.

3️⃣ Chooses an action (e.g., calling a tool, searching for information).

4️⃣ Repeats the process if necessary before generating a final answer.

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper
from langchain.llms import OpenAI

# Initialize LLM
llm = OpenAI(model_name="gpt-4", temperature=0.7)

# Define a math tool
def square_number(n: str):
    return int(n) ** 2

square_tool = Tool(
    name="SquareNumber",
    func=square_number,
    description="Calculates the square of a given number."
)

# Web search tool
search = SerpAPIWrapper()

# Initialize ReAct Agent with tools
react_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Enables reasoning + acting
    tools=[square_tool, search],
    llm=llm,
    verbose=True
)

# Run agent with a query
response = react_agent.run("What is 15 squared and what is the latest news on AI?")
print(response)

"""Expected Behavior of the ReAct Agent
1️⃣ Thinks: Recognizes it needs to calculate 15 squared.
2️⃣ Acts: Calls square_number(15), gets 225.
3️⃣ Thinks again: Determines it needs to search for AI news.
4️⃣ Acts: Uses the web search tool to find the latest AI news.
5️⃣ Responds: "15 squared is 225. As for the latest AI news, here’s what I found: [latest AI news]"""

#13. Creating a Math Solving Agent with LangGraph

Key Ideas:

Math Solving Agents: Use specific chains or agents to process and solve math problems.

LangGraph: Can be used to visualize or structure the steps of reasoning.

Example Concept:
You could design an agent that first parses a math problem, then uses a chain to solve it step by step, and finally returns the answer. (Note: LangGraph is an emerging tool; check for free, community-supported versions or demos.)



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langgraph.graph import StateGraph, END
import operator

# Initialize LLM (Use OpenAI or any free alternative like GPT4All)
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# Step 1: Function to identify the type of math problem
def classify_math_problem(state):
    problem = state["math_problem"]
    if any(op in problem for op in ["+", "-", "*", "/"]):
        return "basic_math"
    return "complex_math"

# Step 2: LLM Chain for Basic Math Problems
def solve_basic_math(state):
    problem = state["math_problem"]
    response = llm.predict(f"Calculate the result: {problem}")
    return {"solution": response}

# Step 3: Python Function for Complex Math Problems
def solve_complex_math(state):
    problem = state["math_problem"]
    try:
        result = eval(problem, {"__builtins__": None}, {"pow": pow, "abs": abs, "sqrt": operator.pow})
        return {"solution": str(result)}
    except Exception as e:
        return {"solution": f"Error: {e}"}

# Step 4: Create a Graph
graph = StateGraph()

# Define nodes
graph.add_node("classify", classify_math_problem)
graph.add_node("basic_math", solve_basic_math)
graph.add_node("complex_math", solve_complex_math)

# Define edges (decision-making)
graph.add_edge("classify", "basic_math", condition=lambda state: state["math_type"] == "basic_math")
graph.add_edge("classify", "complex_math", condition=lambda state: state["math_type"] == "complex_math")
graph.add_edge("basic_math", END)
graph.add_edge("complex_math", END)

# Compile the graph
graph.set_entry_point("classify")
executor = graph.compile()

# Step 5: Run the Agent
input_problem = {"math_problem": "3 * (5 + 2) - 8 / 4"}
solution = executor.invoke(input_problem)
print("Solution:", solution["solution"])

# Expected Behavior
# 1️⃣ Classifies the problem
# 2️⃣ Routes it to the correct solver
# 3️⃣ Returns the computed solution

#14. Custom Tool Creation for Agents


Key Ideas:

Custom Tools: Extend the capabilities of your agents by creating tools that can perform specialized tasks (e.g., web scraping, calculations, data retrieval).

Integration: Tools are integrated into agents so that the agent can “call” them as needed.

Practice: Identify common tasks in your application and wrap them into functions that can be invoked by your agent.

1. What Are Custom Tools?

A tool is a function wrapped in LangChain’s Tool class, which allows the agent to call it when needed.

👉 Example Use Cases:

✅ Scraping web pages

✅ Querying a database

✅ Performing calculations

✅ Interacting with APIs

How to Create and Use a Custom Tool

Let’s build an agent with a custom tool that:

🔹 Scrapes a website for a page title

🔹 Retrieves weather information

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.llms import OpenAI
import requests
from bs4 import BeautifulSoup

# Initialize LLM
llm = OpenAI(model_name="gpt-4", temperature=0.7)

# Custom Tool 1: Web Scraper (Extracts page title)
def scrape_page_title(url: str):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.title.string if soup.title else "No title found"
    except Exception as e:
        return f"Error: {e}"

scraper_tool = Tool(
    name="WebScraper",
    func=scrape_page_title,
    description="Extracts the title of a webpage given a URL."
)

# Custom Tool 2: Fetch Weather (Dummy API Call)
def get_weather(city: str):
    # Simulating an API response
    weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Rainy, 18°C",
        "Tokyo": "Cloudy, 20°C"
    }
    return weather_data.get(city, "Weather data not available")

weather_tool = Tool(
    name="WeatherInfo",
    func=get_weather,
    description="Gets the current weather for a given city."
)

# Initialize the Agent with Custom Tools
agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[scraper_tool, weather_tool],
    llm=llm,
    verbose=True
)

# Run the agent with a query
query = "Get the title of https://www.wikipedia.org/ and the weather in Tokyo."
response = agent.run(query)
print(response)
